# 🧩 Titanic Dataset Preprocessing Example
### Steps: Data Cleaning, Encoding, Scaling, PCA, and Feature Selection

In [25]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2
import seaborn as sns
import matplotlib.pyplot as plt

# Load dataset
data = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 1️⃣ Data Exploration

In [26]:
data.info()
data.describe()
data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


## 2️⃣ Handle Missing Data

In [27]:
print(data.columns)


if 'Age' in data.columns:
    data['Age'] = data['Age'].fillna(data['Age'].median())

if 'Embarked' in data.columns:
    data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])

if 'Cabin' in data.columns:
    data = data.drop('Cabin', axis=1)


print(data.isnull().sum())


Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


## 3️⃣ Encode Categorical Variables

In [28]:
le = LabelEncoder()
data['Sex'] = le.fit_transform(data['Sex'])
data = pd.get_dummies(data, columns=['Embarked'], drop_first=True)
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,False,True
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,False,False
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,False,True
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,False,True
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,False,True


## 4️⃣ Feature Scaling

In [29]:
scaler = StandardScaler()
num_features = ['Age', 'Fare']
data[num_features] = scaler.fit_transform(data[num_features])
data[num_features].head()

,Age,Fare
0,-0.565736,-0.502445
1,0.663861,0.786845
2,-0.258337,-0.488854
3,0.433312,0.420730
4,0.433312,-0.486337


## 5️⃣ PCA (Dimensionality Reduction)

In [30]:
numeric_data = data[['Age', 'Fare', 'Pclass', 'Sex']]
pca = PCA(n_components=2)
pca_features = pca.fit_transform(numeric_data)
pca_df = pd.DataFrame(pca_features, columns=['PC1', 'PC2'])
pca_df.head()

,PC1,PC2
0,-1.019178,-0.118952
1,1.625130,-0.024740
2,-0.807835,0.012068
3,1.271965,-0.021865
4,-0.532838,0.706649


## 6️⃣ Feature Selection

In [31]:
from sklearn.feature_selection import SelectKBest, chi2
import numpy as np


X = data.drop(['Survived', 'Name', 'Ticket', 'PassengerId'], axis=1, errors='ignore')
y = data['Survived']

X = X.select_dtypes(include=[np.number])


X = X - X.min()


selector = SelectKBest(score_func=chi2, k=5)
X_new = selector.fit_transform(X, y)

selected_features = X.columns[selector.get_support()]
print("✅ Selected Features:", list(selected_features))


✅ Selected Features: ['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare']


## ✅ Summary of Transformations

In [32]:
summary = pd.DataFrame({
    'Step': ['Data Exploration', 'Missing Data Handling', 'Encoding', 'Scaling', 'PCA', 'Feature Selection'],
    'Purpose': ['Understand structure', 'Clean missing values', 'Convert categorical to numeric', 'Normalize numeric scale', 'Reduce dimensionality', 'Keep most important features']
})
summary

,Step,Purpose
0,Data Exploration,Understand structure
1,Missing Data Handling,Clean missing values
2,Encoding,Convert categorical to numeric
3,Scaling,Normalize numeric scale
4,PCA,Reduce dimensionality
5,Feature Selection,Keep most important features
